In [107]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

In [76]:
from utils import train_test_split2, generate_portfolio_inputs
from network import MLP, PortfolioLayer
from config import DATA_PATH, PARA_PATH, PRICE_PATH
from sklearn.model_selection import TimeSeriesSplit
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna import trial

In [3]:
# import data and trained network parameters
features = pd.read_csv(DATA_PATH , index_col='Date')
prices   = pd.read_csv(PRICE_PATH, index_col='Date')
net_params = torch.load(PARA_PATH)['net_state_dict']

In [20]:
X, y = train_test_split2(features, test=False)
pf_feature = generate_portfolio_inputs(X, y)
prices = prices.loc[pf_feature.index]

In [23]:
# calculate log return
lret = np.log(prices/prices.shift()).fillna(0).add(1)
lret['CASH'] = 1.0

,VTI,BNDX,VNQ,VWOB,VGK,VWO,BND,VNQI,CASH
Date,,,,,,,,,
2007-01-18,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
2007-01-19,1.003893,1.000209,1.009938,1.003459,1.010844,1.019229,0.999990,1.011732,1.0
2007-01-22,0.996036,0.999960,0.991559,0.998836,0.995032,0.998956,1.000104,0.996719,1.0
2007-01-23,1.002833,1.000151,1.004726,1.002497,1.006279,1.017090,0.999998,1.008585,1.0
2007-01-24,1.008802,1.000188,1.012943,1.002400,1.007830,1.008946,0.999868,1.007328,1.0
...,...,...,...,...,...,...,...,...,...
2021-08-12,1.002786,1.000173,1.002168,1.002268,0.999714,0.993418,1.000116,0.996284,1.0
2021-08-13,1.000565,1.000518,1.005448,1.004271,1.007559,0.998056,1.003945,1.005232,1.0
2021-08-16,1.000391,1.000000,0.998782,0.999624,0.994586,0.991990,1.000926,1.000337,1.0


In [88]:
def objective(trial: optuna.Trial) -> float:
    # set up the parameter using optuna
    LEARNING_RATE = trial.suggest_float("learning rate",1e-6,5e-3,log=True)
    WEIGHT_DECAY  = trial.suggest_float("weight decay" ,1e-6,3e-2,log=True)
    NUM_EPOCHS = 50
    
    # define dataset
    X = pf_feature.loc[:'2016-02-22'].values
    y = lret.loc['2007-01-19':'2016-02-23'].values
    assert X.shape[0] == y.shape[0], 'dataset not equal'

    # define a portfolio layer
    cv  = TimeSeriesSplit(n_splits=10,test_size=150)
    portfolio = PortfolioLayer(asset_num=8, error_adjusted=True)
    optimizer = optim.Adam(params=portfolio.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    val_losses = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        X_tr, y_tr = torch.Tensor(X[train_idx]), torch.Tensor(y[train_idx])
        X_vl, y_vl = torch.Tensor(X[val_idx]), torch.Tensor(y[val_idx])

        dataset = InvDataset(X_tr,y_tr)
        loader = DataLoader(dataset, batch_size=60, shuffle=False)
        
        for epoch in range(NUM_EPOCHS):
            # training
            for x_batch, y_batch in loader:    
                optimizer.zero_grad()
                weights = portfolio(x_batch)
                loss = sharpe_ratio(weights, y_batch)
                loss.backward()
                optimizer.step()
            
            # at the end of the training, evaluate 
            if epoch + 1 == NUM_EPOCHS:
                with torch.set_grad_enabled(False):
                    weights = portfolio(X_vl)
                    val_loss = -sharpe_ratio(weights, y_vl)
                    val_losses.append(val_loss.detach().item())
                    print(f'Fold: {fold+1}, Validation Sharpe: {val_loss.detach()}')

    return np.mean(val_losses)

In [89]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2022-02-27 00:37:58,159] A new study created in memory with name: no-name-6ad72e49-f9b7-4858-b570-b790b154ad2b


Fold: 1, Validation Sharpe: 0.7874948978424072
Fold: 2, Validation Sharpe: 0.718956708908081
Fold: 3, Validation Sharpe: -0.5849557518959045
Fold: 4, Validation Sharpe: 1.026322841644287
Fold: 5, Validation Sharpe: 1.271044373512268
Fold: 6, Validation Sharpe: -1.0473666191101074
Fold: 7, Validation Sharpe: 1.9474117755889893
Fold: 8, Validation Sharpe: 2.478426694869995
Fold: 9, Validation Sharpe: -0.13503018021583557


[I 2022-02-27 00:38:08,402] Trial 0 finished with value: 0.6848051875829697 and parameters: {'learning rate': 0.00059937733700927, 'weight decay': 0.00017384045942619837}. Best is trial 0 with value: 0.6848051875829697.


Fold: 10, Validation Sharpe: 0.38574713468551636
Fold: 1, Validation Sharpe: 2.8908588886260986
Fold: 2, Validation Sharpe: 0.6254957914352417
Fold: 3, Validation Sharpe: 2.123528003692627
Fold: 4, Validation Sharpe: 2.571291208267212
Fold: 5, Validation Sharpe: -0.24956604838371277
Fold: 6, Validation Sharpe: -0.8826520442962646
Fold: 7, Validation Sharpe: 1.8893156051635742
Fold: 8, Validation Sharpe: 2.695181369781494
Fold: 9, Validation Sharpe: -0.13830456137657166


[I 2022-02-27 00:38:18,677] Trial 1 finished with value: 1.0555031955242158 and parameters: {'learning rate': 0.004737873492187529, 'weight decay': 0.00024888755474496496}. Best is trial 1 with value: 1.0555031955242158.


...

Fold: 10, Validation Sharpe: -0.7069830894470215
Fold: 1, Validation Sharpe: 0.8988043665885925
Fold: 2, Validation Sharpe: 0.5541495084762573
Fold: 3, Validation Sharpe: 0.6196012496948242
Fold: 4, Validation Sharpe: 2.3837509155273438
Fold: 5, Validation Sharpe: -0.2537843883037567
Fold: 6, Validation Sharpe: -1.0483922958374023
Fold: 7, Validation Sharpe: 1.9272955656051636
Fold: 8, Validation Sharpe: 2.580683946609497
Fold: 9, Validation Sharpe: 0.010244106873869896


[I 2022-02-27 00:46:29,415] Trial 48 finished with value: 0.8403143828734756 and parameters: {'learning rate': 0.0013165883944994325, 'weight decay': 1.142304735097067e-05}. Best is trial 47 with value: 1.161547015234828.


Fold: 10, Validation Sharpe: 0.7307908535003662
Fold: 1, Validation Sharpe: 0.7383569478988647
Fold: 2, Validation Sharpe: 0.7406556010246277
Fold: 3, Validation Sharpe: -0.7550042271614075
Fold: 4, Validation Sharpe: 0.7844703197479248
Fold: 5, Validation Sharpe: 1.8826520442962646
Fold: 6, Validation Sharpe: -0.0058286190032958984
Fold: 7, Validation Sharpe: 1.1247082948684692
Fold: 8, Validation Sharpe: 0.12514032423496246
Fold: 9, Validation Sharpe: 0.17410889267921448


[I 2022-02-27 00:46:39,800] Trial 49 finished with value: 0.3551303789019585 and parameters: {'learning rate': 1.4881886084529372e-05, 'weight decay': 1.5670619123099693e-06}. Best is trial 47 with value: 1.161547015234828.


Fold: 10, Validation Sharpe: -1.25795578956604


In [91]:
optuna_params = study.best_params

In [93]:
NUM_EPOCHS = 50

# define dataset
X = pf_feature.loc[:'2016-02-22'].values
y = lret.loc['2007-01-19':'2016-02-23'].values
assert X.shape[0] == y.shape[0], 'dataset not equal'

# define a portfolio layer
cv  = TimeSeriesSplit(n_splits=10,test_size=150)
portfolio = PortfolioLayer(asset_num=8, error_adjusted=True)
optimizer = optim.Adam(params=portfolio.parameters(), lr=optuna_params['learning rate'], weight_decay=optuna_params['weight decay'])

val_losses = []
for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
    X_tr, y_tr = torch.Tensor(X[train_idx]), torch.Tensor(y[train_idx])
    X_vl, y_vl = torch.Tensor(X[val_idx]), torch.Tensor(y[val_idx])

    dataset = InvDataset(X_tr,y_tr)
    loader = DataLoader(dataset, batch_size=60, shuffle=False)
    
    for epoch in range(NUM_EPOCHS):
        # training
        for x_batch, y_batch in loader:    
            optimizer.zero_grad()
            weights = portfolio(x_batch)
            loss = sharpe_ratio(weights, y_batch)
            loss.backward()
            optimizer.step()
        
        # at the end of the training, evaluate 
        if epoch + 1 == NUM_EPOCHS:
            with torch.set_grad_enabled(False):
                weights = portfolio(X_vl)
                val_loss = -sharpe_ratio(weights, y_vl)
                val_losses.append(val_loss.detach().item())
                print(f'Fold: {fold+1}, Validation Sharpe: {val_loss.detach()}')


Fold: 1, Validation Sharpe: 3.142106056213379
Fold: 2, Validation Sharpe: 0.7854117751121521
Fold: 3, Validation Sharpe: 2.201643228530884
Fold: 4, Validation Sharpe: 2.550945281982422
Fold: 5, Validation Sharpe: -0.26443934440612793
Fold: 6, Validation Sharpe: -0.8202694058418274
Fold: 7, Validation Sharpe: 1.8603795766830444
Fold: 8, Validation Sharpe: 2.7362654209136963
Fold: 9, Validation Sharpe: -0.023963045328855515
Fold: 10, Validation Sharpe: -0.43247249722480774


In [104]:
params = {}
params["network_parameters"] = portfolio.state_dict()
params["optuna_parameters"]  = study.best_params

In [109]:
torch.save(params, '/Users/mac/Desktop/PycharmProjects/TAADL/models/portfolio_params.json')